In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import lightgbm as lgb
import torch
import random

import utils

In [2]:
class CFG:
    make_folder = True
    note_num = "StackingOpt/EDA006"
    seed = 42


OUTPUT_DIR = f'H:/study/output/{CFG.note_num}/'

if CFG.make_folder:
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)


# id_all_data の　"observed_max"　を修正する

In [12]:
id_all_data = pd.read_csv("H:\study\preprocessing_data\id_all_data.csv",encoding='shift_jis')

In [37]:
def get_all_files_in_directory(directory_path):
    file_list = []
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

directory_path = r'H:\study\preprocessing_data\1_twoweeks_nv'
all_files = get_all_files_in_directory(directory_path)

In [38]:
id_max_gene = dict()
for file in all_files:
    df = pd.read_csv(file, header=None, index_col=None, names=["year","month","day","time","flag","nv","twoweeksmax"])
    df["generation"] = df["nv"]*df["twoweeksmax"]
    id_max_gene[int(file.split("\\")[-1].split(".")[0])] = df["generation"].max()
    #break

In [42]:
id_all_data["observed_max2"] = id_all_data["id"].map(id_max_gene)

In [48]:
id_all_data.to_csv(OUTPUT_DIR+'id_all_data.csv',index=False)

In [53]:
id_all_data = pd.read_csv("H:\study\output\StackingOpt\EDA006\id_all_data.csv")


In [50]:
tt = pd.read_csv(OUTPUT_DIR+'id_all_data.csv')
print(OUTPUT_DIR+'id_all_data.csv')
tt

H:/study/output/StackingOpt/EDA006/id_all_data.csv


,id,id_lat,id_lng,id_lat_mesh,id_lng_mesh,id_prefecture,pvrate,observed_max,observed_max2
0,6,35.81228,139.72822,35.82,139.72,埼玉県,3.42,3014.2112,NaN
1,9,31.92295,131.44116,31.92,131.44,宮崎県,3.00,NaN,NaN
2,12,35.67967,139.82913,35.68,139.82,東京都,6.02,4502.4124,NaN
3,14,35.09792,136.88363,35.10,136.88,愛知県,4.13,3556.2000,NaN
4,17,35.50329,139.69705,35.50,139.70,神奈川県,2.79,2032.7072,NaN
...,...,...,...,...,...,...,...,...,...
5092,6990000048,36.12519,140.51805,36.12,140.52,茨城県,16.50,14596.1192,14773.4
5093,6990000049,36.12519,140.51805,36.12,140.52,茨城県,14.83,13139.6332,13326.2
5094,6990000050,36.19365,140.34600,36.20,140.34,茨城県,16.24,13545.8344,13627.6
5095,6990000051,36.19365,140.34600,36.20,140.34,茨城県,14.25,12180.4000,12250.8


In [54]:
unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]

In [47]:
id_all_data[id_all_data.id.isin(unique_id)][["observed_max","observed_max2"]].isnull().sum()

observed_max     0
observed_max2    0
dtype: int64

In [58]:
id_all_data["|observed_max-observed_max2|"] = np.abs(id_all_data["observed_max"] - id_all_data["observed_max2"])
id_all_data[id_all_data.id.isin(unique_id)]["|observed_max-observed_max2|"].max()

943.0750000000007

In [60]:
id_all_data[id_all_data.id.isin(unique_id)].sort_values(ascending=False,by="|observed_max-observed_max2|").head(20)

,id,id_lat,id_lng,id_lat_mesh,id_lng_mesh,id_prefecture,pvrate,observed_max,observed_max2,|observed_max-observed_max2|
3072,6000000017,35.69908,140.10408,35.70,140.10,千葉県,16.1700,10151.9250,11095.0,943.0750
3536,6170000123,35.66595,140.09407,35.66,140.10,千葉県,16.5000,13399.7864,13889.0,489.2136
2545,2220000001,35.67731,140.10386,35.68,140.10,千葉県,20.0000,18114.5160,18522.0,407.4840
3842,6620000089,35.77488,140.03051,35.78,140.04,千葉県,15.2658,14929.9524,15265.8,335.8476
3071,6000000016,35.69908,140.10408,35.70,140.10,千葉県,16.1700,10136.6350,10451.8,315.1650
4688,6910000200,35.67766,140.03341,35.68,140.04,千葉県,11.0000,8425.6802,8723.2,297.5198
4705,6910000217,35.71602,140.07559,35.72,140.08,千葉県,17.6000,16116.1912,16411.6,295.4088
2547,2220000003,35.67731,140.10386,35.68,140.10,千葉県,14.7634,14471.6414,14763.4,291.7586
1517,1680000002,35.87808,139.95438,35.88,139.96,千葉県,12.4800,9033.8248,9306.2,272.3752
1516,1680000001,35.87808,139.95438,35.88,139.96,千葉県,12.4800,9043.8348,9316.2,272.3652


# EDA006 オプティカルフローの予測値の欠損値補間を行う為のデータの前処理
* pred_interpolated_mesh_data_201308151500.csvは8月15日15時00分の予測値（30分前に予測している）ことに注意！

In [59]:
start_date = "201308150000"
end_date = "201408010000"
data_list = utils.create_time_series_data(start_date,end_date)

In [4]:
lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)

dt= datetime(2013, 8, 15, 6, 30)
df,pred_data = utils.get_prediction_data_all(lat,lon,dt)

In [5]:
for i,date in enumerate(data_list):
    dt = datetime.strptime(date, '%Y%m%d%H%M')
    df,pred_data = utils.get_prediction_data_all(lat,lon,dt)

    pred_data.sort_values(by=["id_lat_mesh","id_lng_mesh"],inplace=True)

    folderpath = f"{OUTPUT_DIR}{dt.month}月/{dt.month}月{dt.day}日/"
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)    

    filename = f"Pred{dt.year-2000}{dt.month}{dt.day}{dt.hour}.{int(dt.minute/60*10)}_time_sort.csv"
    filename2 = f"Pred{dt.year-2000}{dt.month}{dt.day}{dt.hour}.{int(dt.minute/60*10)}_time_sort2.csv"
    
    pred_data[["id_lat_mesh","id_lng_mesh","pred"]].to_csv(folderpath + filename, header=None, index=None)

    pred_data[["id_lat_mesh","id_lng_mesh","pred"]].replace(np.nan,"NaN").to_csv(folderpath + filename2, header=None, index=None)

    if i%int(len(data_list)*0.1) == 0:
        print(f"{i/len(data_list)*100}_finish")
    #break

0.0_finish
9.996283909327387_finish
19.992567818654774_finish
29.988851727982162_finish
39.98513563730955_finish
49.98141954663694_finish
59.977703455964324_finish
69.9739873652917_finish
79.9702712746191_finish
89.96655518394648_finish
99.96283909327389_finish


# MATLAB(EDA006.m,EDA006_2.m)でオプティカルフローの予測値の欠損値補間を行う

補完したデータを読み込んで、オプティカルフローの予測値と発電量のマージを行う。

In [3]:
lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)

start_date = "201308150000"
end_date = "201408010000"
data_list = utils.create_time_series_data(start_date,end_date)

In [92]:
pred_all_list = []
for i,date in enumerate(data_list):
    dt = datetime.strptime(date, '%Y%m%d%H%M')

    path = f"H:/study/output/StackingOpt/EDA006/{dt.month}月/{dt.month}月{dt.day}日/Pred_interpolated_{dt.year%1000}{dt.month}{dt.day}{dt.hour}.{int(dt.minute/60*10)}0.02_int2.csv"
    pred_interpolated = pd.read_csv(path, header=None, index_col=None)
    pred_interpolated  = pd.DataFrame(zip(lat.to_numpy().reshape(-1),lon.to_numpy().reshape(-1),pred_interpolated.to_numpy().reshape(-1)),columns=["id_lat_mesh", "id_lng_mesh", "pred_interpolated"])
    pred_interpolated.dropna(inplace=True)

    folderpath = f"{OUTPUT_DIR}{dt.month}月/{dt.month}月{dt.day}日/"
    filename = f"Pred{dt.year-2000}{dt.month}{dt.day}{dt.hour}.{int(dt.minute/60*10)}_time_sort.csv"
    pred = pd.read_csv(folderpath + filename, header=None, index_col=None,names=["id_lat_mesh", "id_lng_mesh", "pred"])
    pred.dropna(inplace=True)

    pred_all = pd.merge(pred,pred_interpolated,on=["id_lat_mesh","id_lng_mesh"],how="outer")
    pred_all.to_csv(OUTPUT_DIR+f"pred_all_{date}.csv",index=False)
    
    pred_all["datetime"] = dt
    pred_all_list.append(pred_all)

    if i%int(len(data_list)*0.1) == 0:
        print(f"{i/len(data_list)*100}_finish")


0.0_finish
9.996283909327387_finish
19.992567818654774_finish
29.988851727982162_finish
39.98513563730955_finish
49.98141954663694_finish
59.977703455964324_finish
69.9739873652917_finish
79.9702712746191_finish
89.96655518394648_finish
99.96283909327389_finish


In [3]:
id_all_data = pd.read_csv("H:\study\preprocessing_data\id_all_data.csv",encoding='shift_jis')
id_all_data = utils.get_mesh_round(id_all_data)


In [57]:
id_all_data = pd.read_csv("H:\study\preprocessing_data\id_all_data.csv",encoding='shift_jis')
id_all_data = utils.get_mesh_round(id_all_data)

for i,date in enumerate(data_list):
    dt = datetime.strptime(date, '%Y%m%d%H%M')
    pred_all = pd.read_csv(OUTPUT_DIR+f"pred_all_{date}.csv")
    pred_all = utils.get_mesh_round(pred_all)
    save_file = id_all_data.merge(pred_all[["id_lat_mesh","id_lng_mesh","pred","pred_interpolated"]],on=["id_lat_mesh","id_lng_mesh"],how="left")
    save_file["datetime"] = dt
    save_file[["id","pred","pred_interpolated","datetime"]].to_csv(OUTPUT_DIR+f"pred_id_all_data_{date}.csv",index=False)

    #break

    if i%int(len(data_list)*0.1) == 0:
        print(f"{i/len(data_list)*100}_finish")

0.0_finish
9.996283909327387_finish
19.992567818654774_finish
29.988851727982162_finish
39.98513563730955_finish
49.98141954663694_finish
59.977703455964324_finish
69.9739873652917_finish
79.9702712746191_finish
89.96655518394648_finish
99.96283909327389_finish


In [4]:
date = data_list[0]
save_file = pd.read_csv(OUTPUT_DIR+f"pred_id_all_data_{date}.csv")

In [5]:
save_file

,id,pred,pred_interpolated,datetime
0,6,0.796663,0.79666,2013-08-15 07:00:00
1,9,0.543782,0.54378,2013-08-15 07:00:00
2,12,0.565730,0.56573,2013-08-15 07:00:00
3,14,0.750505,0.75051,2013-08-15 07:00:00
4,17,0.743884,0.74388,2013-08-15 07:00:00
...,...,...,...,...
5092,6990000048,0.655848,0.65585,2013-08-15 07:00:00
5093,6990000049,0.655848,0.65585,2013-08-15 07:00:00
5094,6990000050,0.843789,0.84379,2013-08-15 07:00:00
5095,6990000051,0.843789,0.84379,2013-08-15 07:00:00


In [145]:
id_all_data = pd.read_csv("H:\study\output\StackingOpt\EDA006\id_all_data.csv")

folder_path = "H:/study/preprocessing_data/1_twoweeks_nv"

id_list = []
files = os.listdir(folder_path)
for file in files:
    id_list.append(int(file.split(".")[0]))

common_ids = id_all_data[id_all_data.id.isin(id_list)].id.tolist()
common_ids = [str(num).zfill(10) for num in common_ids]
common_ids_path  = [folder_path+f"/{id}.csv" for id in common_ids]

In [155]:
df_list = []
for i,path in enumerate(common_ids_path) :
    df = pd.read_csv(path, header=None, index_col=None)
    df.columns = ["year","month","day","hour","flag","nv","two_weeks_max"]
    id = int(path.split("/")[-1].split(".")[0])
    df["id"] = int(id)
    df_list.append(df)
    if i%int(len(common_ids_path)*0.1) == 0:
        print(f"{i/len(common_ids_path)*100}_finish")
    if int(id)<0:
        print(path)
    


df = pd.concat(df_list) 
df["year"] += 2000
df["minute"] = df["hour"]%1*60
df["hour"] = df["hour"]//1
df[["hour","minute"]] = df[["hour","minute"]].astype(int)
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
df.drop(["year","month","day","hour","minute"],axis=1,inplace=True)


0.0_finish
9.99184339314845_finish
19.9836867862969_finish
29.975530179445354_finish
39.9673735725938_finish
49.95921696574225_finish
59.95106035889071_finish
69.94290375203916_finish
79.9347471451876_finish
89.92659053833604_finish
99.9184339314845_finish


In [137]:
start_date = "201308150000"
end_date = "201408010000"
data_list = utils.create_time_series_data(start_date,end_date)

save_files = []
for i,date in enumerate(data_list):
    save_file = pd.read_csv('H:/study/output/StackingOpt/EDA006/'+f"pred_id_all_data_{date}.csv")
    save_files.append(save_file)
    if i%int(len(data_list)*0.1) == 0:
        print(f"{i/len(data_list)*100}_finish")
    if save_file.id.min()<0:
        print(date)
    #break
# save_files = pd.concat(save_files)
#save_files.to_csv(OUTPUT_DIR+"save_files.csv",index=False)
save_files2 = pd.concat(save_files)

0.0_finish
9.996283909327387_finish
19.992567818654774_finish
29.988851727982162_finish
39.98513563730955_finish
49.98141954663694_finish
59.977703455964324_finish
69.9739873652917_finish
79.9702712746191_finish
89.96655518394648_finish
99.96283909327389_finish


In [172]:
save_files2 = pd.concat(save_files)

df["datetime"] = pd.to_datetime(df["datetime"])
save_files2["datetime"] = pd.to_datetime(save_files2["datetime"])

In [173]:
df2 = df.merge(save_files2,on=["id","datetime"],how="left")


In [177]:
df2["hour"] = df2["datetime"].dt.hour
df2["minute"] = df2["datetime"].dt.minute

In [184]:
df3 = df2[(df2["hour"]>=6)&(df2["hour"]<=18)].reset_index(drop=True)

In [201]:
unique_datetime = df3.datetime.unique()
for i,(datetime,group) in enumerate(df3.groupby("datetime")):
    date = datetime.strftime('%Y%m%d%H%M')
    group.reset_index(drop=True)
    group.to_csv(OUTPUT_DIR+f"optical_flow_pred_{date}.csv",index=False)
    if i%(int(len(unique_datetime)*0.1)) == 0:
        print(f"{i/len(unique_datetime)}_finish")
    #break

0.0_finish
0.1_finish
0.2_finish
0.3_finish
0.4_finish
0.5_finish
0.6_finish
0.7_finish
0.8_finish
0.9_finish


In [207]:
(df3.loc[~df3.pred.isnull(),"nv"] - df3.loc[~df3.pred.isnull(),"pred"]).abs().mean()

0.17141482276687597

In [209]:

df3.loc[~df3.pred.isnull()] 

,flag,nv,two_weeks_max,id,datetime,pred,pred_interpolated,hour,minute
366,2,0.828,872.6,551,2013-08-15 07:00:00,0.804787,0.80479,7,0
367,2,0.716,1279.2,551,2013-08-15 07:30:00,0.732476,0.73248,7,30
368,2,1.000,1638.0,551,2013-08-15 08:00:00,0.842366,0.84237,8,0
369,2,0.762,1846.6,551,2013-08-15 08:30:00,0.735932,0.73593,8,30
370,2,0.879,2217.8,551,2013-08-15 09:00:00,0.762473,0.76247,9,0
...,...,...,...,...,...,...,...,...,...
43144372,2,0.847,4220.4,6990000051,2014-07-31 16:00:00,0.839462,0.83946,16,0
43144373,2,0.859,2747.8,6990000051,2014-07-31 16:30:00,0.903446,0.90345,16,30
43144374,2,0.617,1970.8,6990000051,2014-07-31 17:00:00,0.772158,0.77216,17,0
43144375,2,0.781,979.8,6990000051,2014-07-31 17:30:00,0.592240,0.59224,17,30


# unique_idのメッシュデータを抽出

In [14]:
import utils
import pandas as pd
import datetime


lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)
id_all_data = pd.read_csv(r"H:\study\preprocessing_data\id_all_data.csv", encoding="shift_jis")

unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
id_data = id_all_data[id_all_data.id.isin(unique_id)].reset_index(drop=True)

dt = datetime.datetime(2013, 8, 15, 7, 0)
df = utils.get_pred_interpolated_mesh(id_data,lat,lon,dt)

In [15]:
from datetime import datetime, timedelta

def create_time_series_data(start_date, end_date):

    start_date = datetime.strptime(start_date, "%Y%m%d%H%M")
    end_date = datetime.strptime(end_date, "%Y%m%d%H%M")
    delta = timedelta(minutes=30)

    data_list = []
    date = start_date
    while date <= end_date:
        if date.hour > 6 and date.hour < 18:
            data_list.append(date.strftime("%Y%m%d%H%M"))
        elif date.hour == 18 and date.minute == 0:
            data_list.append(date.strftime("%Y%m%d%H%M"))
        date += delta
    
    return data_list

start_date = "201308150000"
end_date = "201408010000"

date_range = create_time_series_data(start_date,end_date)
# datetimes = [datetime.strptime(ts, '%Y%m%d%H%M') for ts in timestamps]

In [21]:
mesh_data_list,pred_data_list = [],[]
for i,date in enumerate(date_range):
    dt = datetime.strptime(date, '%Y%m%d%H%M')
    mesh_data = utils.get_pred_interpolated_mesh(id_data,lat,lon,dt)
    mesh_data_list.append(mesh_data)
    
    mesh_data.to_csv(OUTPUT_DIR+f"pred_interpolated_mesh_data_{dt.strftime('%Y%m%d%H%M')}.csv")
    if i%int(len(date_range)*0.1)==0:
        print(f"{i/len(date_range)*100}%_finish")
    #break

0.0%_finish
9.996283909327387%_finish
19.992567818654774%_finish
29.988851727982162%_finish
39.98513563730955%_finish
49.98141954663694%_finish
59.977703455964324%_finish
69.9739873652917%_finish
79.9702712746191%_finish
89.96655518394648%_finish
99.96283909327389%_finish


# 予測値を補間したメッシュデータを抽出

In [6]:
# train_start_date = "201308150000"
# train_end_date = "201407010000"
# test_start_date = "201407010000"
# test_end_date = "201408010000"

train_start_date = "201308150000"
train_end_date = "20130817000"
test_start_date = "201308170000"
test_end_date = "20130818000"

# 時系列の分割設定
n_splits = 2
train_data_list = utils.create_time_series_data(train_start_date,train_end_date)
train_date_list_split = np.array_split(train_data_list, n_splits)
test_dates = utils.create_time_series_data(test_start_date,test_end_date)


train_dates = train_date_list_split[0]
valid_dates = train_date_list_split[1]

date = train_dates[0]
date_list = train_dates


pred_dir = "H:\study\output\StackingOpt\EDA006"
date_list = train_dates

X_data2 = utils.get_pred_interpolated_mesh_data(pred_dir,date_list)



In [7]:
X_data2.shape

(23, 192)

# ↑にならって欠損値補間メッシュデータを抽出

In [24]:
# train_start_date = "201308150000"
# train_end_date = "201407010000"
# test_start_date = "201407010000"
# test_end_date = "201408010000"

train_start_date = "201308150000"
train_end_date = "20130817000"
test_start_date = "201308170000"
test_end_date = "20130818000"

# 時系列の分割設定
n_splits = 2
train_data_list = utils.create_time_series_data(train_start_date,train_end_date)
train_date_list_split = np.array_split(train_data_list, n_splits)
test_dates = utils.create_time_series_data(test_start_date,test_end_date)


train_dates = train_date_list_split[0]
valid_dates = train_date_list_split[1]

date = train_dates[0]
date_list = train_dates

interpolated_dir = "H:\study\output\StackingOpt\EDA005"

X_data,Y_data = utils.get_interpolated_mesh_data(interpolated_dir,date_list)


In [25]:
X_data.shape

(23, 192)

# オプティカルフローの速さのデータを抽出する。

In [4]:
lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)
id_all_data = pd.read_csv(r"H:\study\preprocessing_data\id_all_data.csv", encoding="shift_jis")

unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
id_data = id_all_data[id_all_data.id.isin(unique_id)].reset_index(drop=True)

In [9]:
start_date = "201308150000"
end_date = "201408010000"

date_range = utils.create_time_series_data(start_date,end_date)

date = date_range[0]
dt = datetime.strptime(date, '%Y%m%d%H%M')
flo, flo_lat,flo_lon = utils.get_flo_mesh(id_data,lat,lon,dt)


In [20]:
missing_dates = []
for i,date in enumerate(date_range):
    dt = datetime.strptime(date, '%Y%m%d%H%M')
    flo, flo_lat, flo_lon = utils.get_flo_mesh(id_data,lat,lon,dt)

    flo.to_csv(OUTPUT_DIR+f"flo_data_{dt.strftime('%Y%m%d%H%M')}.csv")
    flo_lat.to_csv(OUTPUT_DIR+f"flo_lat_data_{dt.strftime('%Y%m%d%H%M')}.csv")
    flo_lon.to_csv(OUTPUT_DIR+f"flo_lon_data_{dt.strftime('%Y%m%d%H%M')}.csv")
    if i%int(len(date_range)*0.1)==0:
        print(f"{i/len(date_range)*100}%_finish")
    if flo_lat.isnull().sum().sum()!=0 or flo_lon.isnull().sum().sum()!=0:
        missing_dates.append(date)
print(missing_dates)

0.0%_finish
9.996283909327387%_finish
19.992567818654774%_finish
29.988851727982162%_finish
39.98513563730955%_finish
49.98141954663694%_finish
59.977703455964324%_finish
69.9739873652917%_finish
79.9702712746191%_finish
89.96655518394648%_finish
99.96283909327389%_finish
[]


# 上で作ったデータを抽出

In [4]:
lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)
id_all_data = pd.read_csv(r"H:\study\preprocessing_data\id_all_data.csv", encoding="shift_jis")

unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
id_data = id_all_data[id_all_data.id.isin(unique_id)].reset_index(drop=True)

start_date = "201308150000"
end_date = "201408010000"

date_range = utils.create_time_series_data(start_date,end_date)



In [11]:
date_list = date_range[:10]
flo_dir = 'H:\study\output\StackingOpt\EDA006'
flo_data = utils.get_flo_mesh_data(flo_dir,date_list)
flo_lat_data = utils.get_flo_lat_mesh_data(flo_dir,date_list)
flo_lon_data = utils.get_flo_lon_mesh_data(flo_dir,date_list)

In [8]:
date_list = date_range
flo_data = utils.get_flo_mesh_data(flo_dir,date_list)


# 今までのデータの整合性を確認する。
- オプティカルフローの予測値,補間発電量は一致するが、フローの速さは一致しない。以前、フローの速さの決定木の重要度が低かった気がする。（←呉先輩のデータにミス？）
- フロー = (フロー緯度2乗+フロー経度2乗)**(1/2)


In [75]:
lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)
id_all_data = pd.read_csv(r"H:\study\preprocessing_data\id_all_data.csv", encoding="shift_jis")

unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
id_data = id_all_data[id_all_data.id.isin(unique_id)].reset_index(drop=True)
id_data = utils.get_mesh_round(id_data)

In [152]:
start_date = "201308150000"
end_date = "201408010000"

date_range = utils.create_time_series_data(start_date,end_date)
date = date_range[0]
print(date)

#呉先輩のデータ
df = pd.read_csv(r"H:\study\error_data\kanto_parameter_lambda_0.019_iterate_1000_p3\8月\8月15日\error_138157.0_0.02_0.019_1000_p3.csv", encoding='shift-jis')

#検証したいデータ群
flo_lat,flo_lon = utils.get_flo_mesh(id_data,lat,lon,dt)
mesh_data = utils.get_pred_interpolated_mesh(id_data,lat,lon,dt)

train_dates = date_range[:30]
interpolated_dir = "H:\study\output\StackingOpt\EDA005"
X_train_interpolated, y_train_interpolated = utils.get_interpolated_mesh_data(interpolated_dir,train_dates)

#処理
id_lng_mesh , id_lat_mesh = id_data["id_lng_mesh"].iloc[20], id_data["id_lat_mesh"].iloc[20]

row = lon.iloc[:, 0][lon.iloc[:, 0] == id_lng_mesh].index[0]
col = lat.iloc[0,:][lat.iloc[0,:] == id_lat_mesh].index[0]

df.loc[(df.id_lat_mesh==id_lat_mesh)&(df.id_lng_mesh==id_lng_mesh),["flo_lng","flo_lat","flo","env","nv_meshint","nv"]]


201308150700


,flo_lng,flo_lat,flo,env,nv_meshint,nv
20,0.004155,-0.006855,0.006873,0.73,0.542,0.446
41,0.004155,-0.006855,0.006873,0.73,0.542,0.556
42,0.004155,-0.006855,0.006873,0.73,0.542,0.570
79,0.004155,-0.006855,0.006873,0.73,0.542,0.545
86,0.004155,-0.006855,0.006873,0.73,0.542,0.527
87,0.004155,-0.006855,0.006873,0.73,0.542,0.510
98,0.004155,-0.006855,0.006873,0.73,0.542,0.640


In [153]:
print("flo_lat")
flo_lat.iloc[(flo_lat.index==row),(flo_lat.columns==col)]

flo_lat


,227
518,-0.342765


In [154]:
print("flo_lon")
flo_lon.iloc[(flo_lon.index==row),(flo_lon.columns==col)]

flo_lon


,227
518,0.207729


In [155]:
print("env")
mesh_data.iloc[(mesh_data.index==row),(mesh_data.columns==col)]

env


,227
518,0.72957


In [156]:
y_train_interpolated[0].reshape(flo_lat.shape[0],flo_lat.shape[1])[row-flo_lat.index.max()-1,col-flo_lat.columns.max()-1]

0.542

In [158]:
df["flo"] - np.sqrt((df["flo_lat"]**2+df["flo_lng"]**2)) 

0     -2.057271e-03
1     -3.986184e-04
2     -2.954866e-04
3     -1.848768e-03
4     -7.179927e-07
           ...     
98    -1.143374e-03
99    -1.224577e-04
100   -6.267781e-04
101   -4.560104e-04
102   -4.560104e-04
Length: 103, dtype: float64

In [160]:
np.abs(df["flo"] - np.sqrt((df["flo_lat"]**2+df["flo_lng"]**2))).max()

0.025525917781522366

In [162]:
df[["flo"]].describe()

,flo
count,103.000000
mean,0.009646
std,0.004753
min,0.001676
25%,0.006845
50%,0.009017
75%,0.012911
max,0.021125
